## Mass copy live_data to stbc

03.12.2021

j.angevaare@nikhef.nl

# Goal
Copy data from the XAMS DAQ to stoomboot

In [1]:
from tqdm.notebook import tqdm
import glob
import subprocess
import os
import pymongo
import time

copy data from these two folders on xams:

In [2]:
bases = ['/media/xams/Elements/xams', 
         '/media/xams/Elements/run11']

Let's see how big these folders are (this can take a while!)

In [3]:
for b in bases:
    !ssh xams -t "du -h -d0 $b" 

3,2T	/media/xams/Elements/xams
Connection to 145.102.133.168 closed.
244G	/media/xams/Elements/run11
Connection to 145.102.133.168 closed.


Select folders within the folders to copy the live data of

In [4]:
to_do = []
for b in bases:
    fs = !ssh xams -t "ls $b" 
    to_do += [os.path.join(b, f) for f in fs]
# Filter folders that don't have a lenght of 6 characters:
to_do = [t for t in to_do if len(t.split('/')[-1])==6]

That leaves this many folders to sync:

In [5]:
len(to_do)

2268

The folders all look something like this:

In [6]:
to_do[:5]

['/media/xams/Elements/xams/000736',
 '/media/xams/Elements/xams/000768',
 '/media/xams/Elements/xams/000769',
 '/media/xams/Elements/xams/000770',
 '/media/xams/Elements/xams/000771',
 '/media/xams/Elements/xams/000772',
 '/media/xams/Elements/xams/000773',
 '/media/xams/Elements/xams/000774',
 '/media/xams/Elements/xams/000775',
 '/media/xams/Elements/xams/000776',
 '/media/xams/Elements/xams/000777',
 '/media/xams/Elements/xams/000778',
 '/media/xams/Elements/xams/000779',
 '/media/xams/Elements/xams/000780',
 '/media/xams/Elements/xams/000781',
 '/media/xams/Elements/xams/000782',
 '/media/xams/Elements/xams/000783',
 '/media/xams/Elements/xams/000784',
 '/media/xams/Elements/xams/000785',
 '/media/xams/Elements/xams/000786',
 '/media/xams/Elements/xams/000787',
 '/media/xams/Elements/xams/000788',
 '/media/xams/Elements/xams/000789',
 '/media/xams/Elements/xams/000790',
 '/media/xams/Elements/xams/000791',
 '/media/xams/Elements/xams/000792',
 '/media/xams/Elements/xams/000794',
 

## Starting the sync

Below we will start rsync. We are going to allow max. 25 jobs to run at the same time (fast but not too heavy on the filesystem hopefully).

We are going to do this using `subprocess.Popen` and start new syncs every time there is one of the 25 spots available. Otherwise we need to wait for a bit

In [5]:
sleep = 60
max_running = 25
# Set to True if you want to start, otherwise we will just print but do nothing
do_run = False
running = []
done = 0

# This is where to folders should be copied to
dest_loc = f'/data/xenon/xamsl/live_data'

In [8]:
for source_loc in tqdm(to_do):
    target = os.path.join(dest_loc, source_loc.split('/')[-1])
    command = f'rsync -a -e ssh xams:{source_loc} {dest_loc}/.'
    if os.path.exists(target):
        UserWarning(f'{target} exists')
    else:
        while len(running) > max_running:
            print(f"Sleeping {sleep} sec, to see if new rsync slot available")
            time.sleep(sleep)
            running = [x 
                       for x in running 
                       if x.poll() is None]
            sleep *= 2
            sleep = min(sleep, 300)
        sleep = 10

        print(command)
        if do_run:
            running.append(subprocess.Popen(command.split()))


  0%|          | 0/2268 [00:00<?, ?it/s]

rsync -a -e ssh xams:/media/xams/Elements/xams/000932 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/000933 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/000951 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001362 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001375 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001395 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001398 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001606 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001910 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001967 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001971 /data/xenon/xamsl/live_data/.
rsync -a -e ssh xams:/media/xams/Elements/xams/001973 /data/xenon/xamsl/live